###  Objective
To study the effect of immune infiltration in endocrine treated breast cancer patients. The data is from 36 patients who have been given Anastrazole and saples have been taken from before and after treatment. The dataset contains 15 non-responders and 24 responders.

### Import packages

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

### Import data
* The immune signature was defined by a manual feature selection of the most co-expressed genes (ImSig), identified from the network graph.

In [2]:
# Set Working directory
WD = '//ris-fas1a.roslin.ed.ac.uk/v1anirma/Win7/Desktop/Anastrazole'
os.chdir(WD)
# Import data
exp = pd.read_csv('pre_collapse.txt', delimiter='\t', index_col=0)
sig = pd.read_csv('feature selected genes.txt', delimiter='\t', index_col=0)
response = pd.read_csv('response.txt', delimiter='\t', index_col=0)

### Reduce multiple transcripts into one gene

In [3]:
grouped = exp.groupby(exp.index).max()
grouped = grouped.apply(lambda x: np.power(x,2), axis=1)
# Write out the collapsed DataFrame if necessary
# grouped.to_csv('Post Collapse.txt', sep= "\t")

### Calculate the average expression of signature genes
* The objective is to test if there is a change in the number of immune cells between the responders and non-responders. 